# [The Slice Type](https://doc.rust-lang.org/book/ch04-03-slices.html)

So, slicing.
I knew we'd get to this soon.

I'll be interesting to see how this interacts with the complex ownership rules.

```rust
fn first_word(s: &String) -> usize {
    let bytes = s.as_bytes();

    for (i, &item) in bytes.iter().enumerate() {
        if item == b' ' {
            return i;
        }
    }

    s.len()
}
```

So this seems to create a byte array and look though it for a space.

I'm guessing `as_bytes()` is implicitly passing ownership to the caller.

As much as I don't like looking at UTF-8 as bytes, it should be reliable.

The highest bit will be set for multipart characters so it will not match space.

A few interesting things:
 * A type capable of iterating can not automatically be used in a `for` loop, it needs the `.iter()` method.
    * Python has spoiled me a lot in this regard. Their loops just work without the additional method call.
 * Even that isn't enough, it **also** needs `.enumerate()`. I don't know why.
    * Ah I see it's keeping track of the loop position as well as item for the return.
 * Why do we need the 'b' in `b' '`, is there anything else a single quote could mean?
 * It seems `i` is automatically assigned as `usize`.
    * I don't think the loop index can be anything else?

## [String Slices](https://doc.rust-lang.org/book/ch04-03-slices.html#string-slices)

They have a pretty standard slicing syntax:
```rust
    let s = String::from("hello world");
    let hello = &s[0..5];
    let world = &s[6..11];
```

A thoughts:
 * I assume this is a byte index. It doesn't look unicode safe.
    * Slicing after an ASCII space should be fine.
    * Slicing arbitrarily it likely to cause UTF-8 decoding errors.
    * The tutorial does later acknowledge this., but solutions aren't until chapter eight.
    * A unicode BOM would likely be accidently included with the previous function.
 * Range operators are `[first...(last+1)]`, so `2..5` = `2, 3, 4`.
    * Can also do standard slice tricks like `s[n..]` meaning everything after and including the nth position.
    * Also `s[..n]` for everything before the nth position.
    * Strangly also `s[..]` for a slice of the whole thing.
        * Is that different from a reference???
        * Seems to be as slices have a different type.
    * Methods like `.clear()` require mutability.
 * Slices operate as a kind of view on the object.
 * They have a different type, they're recorded as `&str` rather than the `&String` type.
 
I understand the benefits of this, though maybe they didn't need to boast at the end of the section.


## [String Literals as Slices](https://doc.rust-lang.org/book/ch04-03-slices.html#string-literals-as-slices)
String literal are just slices from the binary!

This seems to be the reason why user generated slices from `String` have the same `str` type.

Which means modifying slices would need a copy in a new memory location initialize with `String::from()` right???

```rust
fn first_word(s: &String) -> &str { }

fn first_word(s: &str) -> &str { }
```

The tutorial says the second function signature would be more versatile.

Often we'd be working with the `str` type and `String` can be easily converted with slicing.
They gave an example:
```rust
    let my_string = String::from("hello world");
    let word = first_word(&my_string[0..6]);
    let word = first_word(&my_string[..]);
```
Seems pretty self explainitory at this point.

I assume the allocated of slices happens outside the heap since slices would be a small predictable pointer structure.




## [Other Slices](https://doc.rust-lang.org/book/ch04-03-slices.html#other-slices)

A slice can also be part of an array. 

```rust
let a = [1, 2, 3, 4, 5];
let slice = &a[1..3];
assert_eq!(slice, &[2, 3]);
```

This is just an element pointer an length.

The `..` syntax from before returns. 

It's nice to do this with one operator to keep it as a single parameter unit.

The start is element one with zero based indexing making it the number 2.

Element three would be 4, but since the syntax stops before that it just captures element one and two.

Resulting in `&[2, 3]` with a type of `&[i32]`.

So do the brackets in the type indicate a slice, or would any reference to a number do that?


## Practice Program
Can't forget to do these!

### Initial Attempt
I wrote a quick "detect threes" program to test this out. 
```rust
fn main() {
    let prompt = "The elevator is on the third floor, three rooms down the hallway next to a trio of cheeses.";
    loop {
        let three = detect_three(&mut prompt);
        if three == "" { break }
        println!("Detected three: {three}");
    }

}

fn detect_three(text: &mut str) -> str {
    let length = text.len();
    let threes = ["third", "three", "trio"];
    
    for offset in 0..length {
        for option in threes.iter() {
            if text[offset..offset+option.len()] == option {
                text = text[offset..];
                return option;
            }
        }
    }
    return ""
}
```

There are... a lot of errors.

### Error #1:
Mixed up `str` and `&str` in function return.

```fn detect_three(text: &mut str) -> str {```

->```fn detect_three(text: &mut str) -> &str {```

It's a reference not a literal.

### Error #2:
`option` is a double pointer???

```if text[offset..offset+option.len()] == option {```

->```if text[offset..offset+option.len()] == **option {```

I suppose it's a reference from `.iter()` and again becuase it's from a str array.

### Error #3:
This raised an error because it's not mutable.
```                text = &text[offset..];```

How were we supposed to define an argument modifiable by the caller.

Perhaps the answer is don't.

I ended up deciding this is probably just bad programming and opted to instead return a tuple.

It contains the type of three found and the current position.

```
fn detect_three(text: &str) -> (&str, &str) {
    let length = text.len();
    let threes = ["third", "three", "trio"];
    
    for offset in 0..length {
        for option in threes.iter() {
            if text[offset..offset+option.len()] == **option {
                return (option, &text[..offset+option.len()]);
            }
        }
    }
    return ("", "")
}
```

It'd probably be better to raise an error than this nasty magic blank, but I haven't learnt that.

It took a bit of head scratching on the difference between a mutable object and mutable reference to a constant object.

But I got it to work like this:

```rust
fn main() {
    let prompt = &mut "The elevator is on the third floor, three rooms down the hallway next to a trio of cheeses.";
    let three = &mut "";
    loop {
        (*three, *prompt) = detect_three(prompt);
        if *three == "" { break }
        println!("Detected three: {three}");
    }

}

fn detect_three(text: &str) -> (&str, &str) {
    let length = text.len();
    let threes = ["third", "three", "trio"];
    
    for offset in 0..length {
        for option in threes.iter() {
            if length - offset < option.len() { continue }
            if text[offset..offset+option.len()] == **option {
                return (option, &text[offset+option.len()..]);
            }
        }
    }
    return ("", "")
}
```

So know when to use your `mut`s.

I had to add a bounds check to avoid a panic at the end. 

Certainly more unit testing would be nice.

